In [3]:
%cd ..

w:\python\HLandscape


In [2]:
# import autopipimport 
import requests
import re
import pandas as pd
import pyperclip
from pyzotero import zotero

In [12]:
# def extract_dois(url):
#   """Extracts DOIs from a URL.

#   Args:
#     url: A URL.

#   Returns:
#     A list of DOIs.
#   """

#   response = requests.get(url)
#   text = response.text
#   dois = re.findall(r'\b10\.\d{4,9}/[-.;()/:\w]+', text)
#   return dois
# add_str = lambda x: 'https://doi.org/' + str(x)
# def main():
#   url = pyperclip.paste()
#   dois = extract_dois(url)
#   links = list(map(add_str, dois))
#   df = pd.DataFrame({"DOI": links})
#   df.to_csv("dois.csv", index=False)
#   df['DOI'].to_clipboard(index=False, header=False)
  

# if __name__ == "__main__":
#   main()


In [3]:
def extract_dois(url):
  """Extracts DOIs from a URL.

  Args:
    url: A URL.

  Returns:
    A list of DOIs.
  """

  response = requests.get(url)
  text = response.text
  dois = re.findall(r'\b10\.\d{4,9}/[-.;()/:\w]+', text)
  return dois
add_str = lambda x: 'https://doi.org/' + str(x)

def main():
  url = pyperclip.paste()
  dois = extract_dois(url)
  links = list(map(add_str, dois))
  df = pd.DataFrame({"DOI": links})
  df.to_csv("dois.csv", index=False)
  df['DOI'].to_clipboard(index=False, header=False)
  return df['DOI']

def add_items_to_zotero_by_doi(doi_list, library_id, api_key, collection_id):
    # 初始化 Pyzotero 对象
    zot = zotero.Zotero(library_id, "user", api_key)
    
    for doi_url in doi_list:
        # 去除 "https" 前缀
        doi = doi_url.replace("https://doi.org/", "")
        
        # 使用 CrossRef API 获取元数据
        crossref_url = f"https://api.crossref.org/works/{doi}"
        response = requests.get(crossref_url)
        metadata = response.json()['message']
        
        # 创建新项的模板
        new_item_template = zot.item_template('journalArticle')  # 以期刊文章为例
        new_item_template['title'] = metadata.get('title')[0]
        new_item_template['DOI'] = doi
        new_item_template['creators'] = [{"creatorType": "author", "name": author['family']} for author in metadata.get('author', [])]
        
        # 提交新项
        new_item = zot.create_items([new_item_template])
        
        # 将新项添加到特定集合
        if new_item['successful']:
            item_keys = list(new_item['successful'].values())
            for item_key in item_keys:
                try:
                    # 确保项实际存在
                    item_data = zot.item(item_key)
                    if item_data:
                        payload = {
                            'key': item_key,
                            'version': item_data['version']
                        }
                        zot.addto_collection(collection_id, payload)
                        print(f"Item successfully added to collection with DOI: {doi}")
                    else:
                        print(f"Item with key {item_key} not found.")
                except Exception as e:
                    print(f"An error occurred: {e}")
        else:
            print(f"Failed to add item with DOI: {doi}")


In [16]:
if __name__ == "__main__":
  doi_list = main()
  # library_id = '9769982'
  # api_key = 'fw2EPYr4Dby6PulXbrftoSZ2'
  # collection_id = "SILYZPVI"
  # add_items_to_zotero_by_doi(list(doi_list), library_id, api_key, collection_id)
  doi_list_string = '\n'.join(doi_list.astype(str))
  pyperclip.copy(doi_list_string)


In [18]:
list(doi_list_string)

[]

In [10]:
pyperclip.copy(doi_list)